#Homework 2

## Task 1

We calculate the metrics using formulas from the lecture. We assume that in the red group the random choice of enrollment is independent of the real need. We get the following results:

In [45]:
display(analysis_results.T)

,Blue,Red
Demographic parity,0.65,0.5
Equal opportunity,0.75,0.5
Positive predictive value,0.92,0.5
Negative predictive value,0.43,0.5


It can be seen that the decision rule is not fair by any means.

## Task 2

We use the [adult income dataset](https://www.kaggle.com/datasets/wenruliu/adult-income-dataset). We transform the age feature to age groups:

*   <18
*   18-29
*   30-39
*   40-49
*   etc.

Moreover, we encode all the categorical features using one-hot encoding.
### XGBoost
We use XGBoost as a model for classification. Then we evaluate the fairness metrics:



In [46]:
display(boost_results_df)

,<18,18-29,30-39,40-49,50-59,60-69,70-79,80-89,90-99
number of samples,595.0,13920.00,12929.00,10724.00,6619.00,3054.00,815.00,131.00,55.00
demographic parity,0.0,0.05,0.27,0.37,0.38,0.26,0.18,0.12,0.24
equal opportunity,NaN,0.83,0.81,0.83,0.82,0.86,0.92,0.91,0.82
positive predictive value,NaN,0.46,0.65,0.76,0.73,0.65,0.67,0.62,0.69
negative predictive value,1.0,0.99,0.95,0.91,0.90,0.96,0.99,0.99,0.95


As we can see, we have few data for the youngest and the oldest. Apart from that we can see that demographic parity varies a lot between different age groups, equal opportunity is quite similar in the 4 most populated groups and does not vary much between all the groups. Positive predictive value has quite similar values, apart from young groups and negative predictive value is very similar between all the groups.

### Random forest
Later, we use a different model, namely a Random Forest. We get the following results:

In [48]:
display(results_random_forest_df)

,<18,18-29,30-39,40-49,50-59,60-69,70-79,80-89,90-99
number of samples,595.0,13920.00,12929.00,10724.00,6619.00,3054.00,815.00,131.00,55.00
demographic parity,0.0,0.05,0.27,0.37,0.38,0.26,0.18,0.12,0.24
equal opportunity,NaN,0.98,1.00,0.98,0.99,0.96,1.00,1.00,0.80
positive predictive value,NaN,0.06,0.12,0.18,0.15,0.18,0.22,0.38,0.31
negative predictive value,1.0,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.98


In this case we get very similar demographic parity measures. Equal opportunity measures are much more close. Positive predictive value is quite similar in the middle age groups, while it differs much in younger and older groups. Negative predictive value is still very similar.

### Bias mitigation
Later, we applied sample weighting to mitigate bias, the weight were inversely proportional to the share of group in the whole dataset. We used the XGBoost model as before. Here are the results:

In [54]:
display(results_bias_mitigation_df)

,<18,18-29,30-39,40-49,50-59,60-69,70-79,80-89,90-99
number of samples,595.0,13920.00,12929.00,10724.00,6619.00,3054.00,815.00,131.00,55.00
demographic parity,0.0,0.05,0.27,0.37,0.38,0.26,0.18,0.12,0.24
equal opportunity,NaN,0.79,0.80,0.83,0.82,0.90,0.99,1.00,1.00
positive predictive value,NaN,0.43,0.64,0.75,0.74,0.72,0.92,1.00,1.00
negative predictive value,1.0,0.99,0.94,0.91,0.90,0.97,1.00,1.00,1.00


This bias mitigation method did not help with demographic parity. It even worsened equal opportunity measure. It did not change much predictive values. It seems that this bias mitigation technique is not suited for this feature, dataset or model.

### Performance
We compared the performance of those three methods. The results are as following:

In [57]:
display(performance_df)

,accuracy,balanced accuracy,F1 score,RoC AuC score
XGBoost,0.89,0.82,0.75,0.82
Random forest,0.80,0.57,0.26,0.57
XGBoost with case weighting,0.90,0.84,0.77,0.84


As we can see the model with the best fairness measures (Random Forest) is simulatnously the model with the worst balanced accuracy, F1 score and RoC AuC score. We can also note that sample weighting did improve the performance of XGBoost.

# Appendix

## Task 1

In [1]:
import pandas as pd
import numpy as np

In [2]:
blue_group_results = np.array([[60, 5], [20, 15]])

def demographic_parity(results):
    return results[0].sum() / results.sum()

def equal_opportunity(results):
    return results[0, 0] / results[:, 0].sum()

def predictive_rate_parity(results):
    positive_predictive_value = results[0, 0] / results[0].sum()
    negative_predictive_value = results[1, 1] / results[1].sum()
    return positive_predictive_value, negative_predictive_value

def positive_predictive_value(results):
    return results[0, 0] / results[0].sum()

def negative_predictive_value(results):
    return results[1, 1] / results[1].sum()


In [3]:
print(blue_group_results)
print(demographic_parity(blue_group_results))
print(equal_opportunity(blue_group_results))
print(predictive_rate_parity(blue_group_results))

[[60  5]
 [20 15]]
0.65
0.75
(0.9230769230769231, 0.42857142857142855)


Assuming the enrollment decision is independent to actual need in the red group:

In [4]:
red_group_results = 25 * np.ones((2, 2))

In [5]:
print(red_group_results)
print(demographic_parity(red_group_results))
print(equal_opportunity(red_group_results))
print(predictive_rate_parity(red_group_results))

[[25. 25.]
 [25. 25.]]
0.5
0.5
(0.5, 0.5)


In [6]:
analysis_results = pd.DataFrame({
    'Demographic parity': [demographic_parity(results) for results in [blue_group_results, red_group_results]],
    'Equal opportunity': [equal_opportunity(results) for results in [blue_group_results, red_group_results]],
    'Positive predictive value': [predictive_rate_parity(results)[0] for results in [blue_group_results, red_group_results]],
    'Negative predictive value': [predictive_rate_parity(results)[1] for results in [blue_group_results, red_group_results]]
}, index=['Blue', 'Red'])

pd.set_option("display.precision", 2)
display(analysis_results.T)

,Blue,Red
Demographic parity,0.65,0.5
Equal opportunity,0.75,0.5
Positive predictive value,0.92,0.5
Negative predictive value,0.43,0.5


The metrics are not close to each other, this decision method is not good.

## Task 2

### Setup

In [7]:
!pip install xgboost

In [8]:
!unzip drive/MyDrive/adult-income-dataset.zip

Archive:  drive/MyDrive/adult-income-dataset.zip
  inflating: adult.csv               


In [9]:
df = pd.read_csv('adult.csv')

def group_by_age(age):
    if age < 18:
        return '<18'
    elif age <= 29:
        return '18-29'
    else:
        d = age // 10
        return f'{d}0-{d}9'

df['age_grouped'] = df.age.apply(group_by_age)

numeric_variables = ['fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week']
categorical_variables = ['age_grouped', 'workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender', 'native-country']

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

one_hot_enc = OneHotEncoder(drop='first')
X_categorical = one_hot_enc.fit_transform(df[categorical_variables]).toarray()
X_numeric = df[numeric_variables].to_numpy()
X = np.concatenate((X_numeric, X_categorical), axis=1)
y = (df.income == '>50K').astype(np.float32).to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [13]:
from sklearn.metrics import confusion_matrix, accuracy_score

def prepare_results_df(predictions_):
    results = pd.DataFrame(one_hot_enc.inverse_transform(X_categorical), columns=categorical_variables)
    results['true_value'] = y
    results['predicted_value'] = predictions_

    results_by_group = dict()

    for age in results.age_grouped.unique():
        results_in_group = results[results.age_grouped == age]
        confusion_matrix_group = confusion_matrix(results_in_group.true_value, results_in_group.predicted_value, labels=[1, 0])
        dict_group = dict()
        dict_group['number of samples'] = confusion_matrix_group.sum()
        for metric_func in [demographic_parity, equal_opportunity, positive_predictive_value, negative_predictive_value]:
            dict_group[metric_func.__name__.replace('_', ' ')] = metric_func(confusion_matrix_group)

        results_by_group[age] = dict_group

    results_df = pd.DataFrame(results_by_group)
    return results_df[['<18', '18-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89', '90-99']]

### XGBoost

In [27]:
from xgboost import XGBClassifier

boost = XGBClassifier().fit(X, y)
predictions_boost = boost.predict(X)

In [ ]:
boost_results_df = prepare_results_df(predictions_boost)

In [29]:
display(boost_results_df)

,<18,18-29,30-39,40-49,50-59,60-69,70-79,80-89,90-99
number of samples,595.0,13920.00,12929.00,10724.00,6619.00,3054.00,815.00,131.00,55.00
demographic parity,0.0,0.05,0.27,0.37,0.38,0.26,0.18,0.12,0.24
equal opportunity,NaN,0.83,0.81,0.83,0.82,0.86,0.92,0.91,0.82
positive predictive value,NaN,0.46,0.65,0.76,0.73,0.65,0.67,0.62,0.69
negative predictive value,1.0,0.99,0.95,0.91,0.90,0.96,0.99,0.99,0.95


### Random forest

In [31]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(max_depth=4, random_state=42).fit(X, y)
predictions_random_forest = random_forest.predict(X)

In [ ]:
results_random_forest_df = prepare_results_df(predictions_random_forest)

In [47]:
display(results_random_forest_df)

,<18,18-29,30-39,40-49,50-59,60-69,70-79,80-89,90-99
number of samples,595.0,13920.00,12929.00,10724.00,6619.00,3054.00,815.00,131.00,55.00
demographic parity,0.0,0.05,0.27,0.37,0.38,0.26,0.18,0.12,0.24
equal opportunity,NaN,0.98,1.00,0.98,0.99,0.96,1.00,1.00,0.80
positive predictive value,NaN,0.06,0.12,0.18,0.15,0.18,0.22,0.38,0.31
negative predictive value,1.0,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.98


### XGBoost with bias mitigation

In [51]:
from sklearn.utils.class_weight import compute_sample_weight
from collections import Counter

age_groups_weights = {k: len(df)/v for k, v in dict(Counter(df.age_grouped)).items()}
sample_weights = compute_sample_weight(age_groups_weights, df.age_grouped)

boost_with_bias_mitigation = XGBClassifier().fit(X, y, sample_weight=sample_weights)
predictions_bias_mitigation = boost_with_bias_mitigation.predict(X)

results_bias_mitigation_df = prepare_results_df(predictions_bias_mitigation)

<ipython-input-2-e2f093cb8b73>:7: RuntimeWarning: invalid value encountered in long_scalars
  return results[0, 0] / results[:, 0].sum()
<ipython-input-2-e2f093cb8b73>:15: RuntimeWarning: invalid value encountered in long_scalars
  return results[0, 0] / results[0].sum()


In [52]:
display(results_bias_mitigation_df)

,<18,18-29,30-39,40-49,50-59,60-69,70-79,80-89,90-99
number of samples,595.0,13920.00,12929.00,10724.00,6619.00,3054.00,815.00,131.00,55.00
demographic parity,0.0,0.05,0.27,0.37,0.38,0.26,0.18,0.12,0.24
equal opportunity,NaN,0.79,0.80,0.83,0.82,0.90,0.99,1.00,1.00
positive predictive value,NaN,0.43,0.64,0.75,0.74,0.72,0.92,1.00,1.00
negative predictive value,1.0,0.99,0.94,0.91,0.90,0.97,1.00,1.00,1.00


### Performance

In [43]:
from sklearn.metrics import f1_score, roc_auc_score, balanced_accuracy_score

performance = dict()

for method, predictions in zip(
    ['XGBoost', 'Random forest', 'XGBoost with case weighting'],
    [predictions_boost, predictions_random_forest, predictions_bias_mitigation]):

    performance[method] = {
        'accuracy': accuracy_score(y, predictions),
        'balanced accuracy': balanced_accuracy_score(y, predictions),
        'F1 score': f1_score(y, predictions),
        'RoC AuC score': roc_auc_score(y, predictions),
    }

performance_df = pd.DataFrame(performance).T

In [56]:
display(performance_df)

,accuracy,balanced accuracy,F1 score,RoC AuC score
XGBoost,0.89,0.82,0.75,0.82
Random forest,0.80,0.57,0.26,0.57
XGBoost with case weighting,0.90,0.84,0.77,0.84
